In [ ]:
 
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import functools
from PIL import Image
import cv2
from numpy import asarray
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.python.framework import ops
import timeit
from pathlib import Path

In [ ]:
#Comando para linkar colab e drive!!
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#Celula pra ver GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
import pathlib
 

class dataset_Helper():
 
    def __init__(self, rootDir, trainPerc, testPerc, valPerc):
        '''Create dataset helper for each dose reduction factor'''
        
        if trainPerc + testPerc + valPerc != 1.0:
            raise ValueError('Percentages should sum to 1.')
           
        rootDir2Read = pathlib.Path(rootDir)
        
        imgFiles = list(rootDir2Read.glob('*SinoMetal*.tiff'))
        
        
 
        trainAmount =  np.round(len(imgFiles) * trainPerc).astype('int') 
        testAmount = np.round(len(imgFiles) * testPerc).astype('int') 
        
        imgMetal = []
        
        for imgFile in imgFiles:
            
            imgFile = str(imgFile)
            
            
 
            if 'SinoMetal' in imgFile:
            # if imgFile[-14:-5] == 'SinoMetal':
 
                imgMetal.append(imgFile)
                
        self.noiseData_train = imgMetal[0:trainAmount]
        self.noiseData_test  = imgMetal[trainAmount:trainAmount+testAmount]
        self.noiseData_val   = imgMetal[trainAmount+testAmount:]
                               
    def shuffle_dataset(self):
        '''Shuffle the TF Dataset on each epoch'''
        
        perm = list(range(len(self.noiseData_train)))
        random.shuffle(perm)
        shuffle_array = [self.noiseData_train[index] for index in perm]
        self.noiseData_train = shuffle_array
        
    def create_train_dataset_generator(self, imgBatchSize):
        
        noiseDataset = self.noiseData_train
        
        return self.__create_dataset_generator(noiseDataset, imgBatchSize)
    
    def create_test_dataset_generator(self, imgBatchSize):
        
        noiseDataset = self.noiseData_test
        
        return self.__create_dataset_generator(noiseDataset, imgBatchSize)
    
    def create_val_dataset_generator(self, imgBatchSize):
        
        noiseDataset = self.noiseData_val
        
        return self.__create_dataset_generator(noiseDataset, imgBatchSize)
        
    def __create_dataset_generator(self, noiseDataset, imgBatchSize):
        ''' Create a generator to run over the dataset (train, test or val). 
            The generator get bathches of dicom images'''
           
        for x in range(0,len(noiseDataset),imgBatchSize):
                
            batch_noiseDcmFile = noiseDataset[x:x+imgBatchSize]
            normal_ROIs_dataset = []
            noise_ROIs_dataset = []
 
            for noiseDcmFile in batch_noiseDcmFile:
                split = noiseDcmFile.split('/')
                split[-1] = split[-1].split('_')[0] + "_SinoOriginal.tiff"
                normalDcmFile = "/".join(split)
 
                # Get noise and normal ROIs from Dicom data
                noise_ROIs  = plt.imread(noiseDcmFile).astype('float32') / 255
                normal_ROIs = plt.imread(normalDcmFile).astype('float32') / 255
 
                shape = noise_ROIs.shape
 
                noise_ROIs = noise_ROIs.reshape(1, *shape, 1)
                normal_ROIs = normal_ROIs.reshape(1, *shape, 1)
                # Append data to the ROI dataset list
                normal_ROIs_dataset.append(normal_ROIs)
                noise_ROIs_dataset.append(noise_ROIs)
         
            # Stack all the ROIs in one np array
            normal_ROIs_dataset =  np.concatenate(normal_ROIs_dataset, axis=0)
            noise_ROIs_dataset =  np.concatenate(noise_ROIs_dataset, axis=0)
            yield (noise_ROIs_dataset, normal_ROIs_dataset)
    
 
dataset = dataset_Helper('/content/drive/My Drive', trainPerc=0.8, testPerc=0.1, valPerc=0.1) 
 
roiSize = 180

imgBatchSize_train  = 16     # Amount of images to read from train dicom folder
imgBatchSize_val = 128     # Amount of images to read from val dicom folder
imgBatchSize_test = 128    # Amount of images to read from val dicom folder
batch_size = 5
 
# Create a generator to run over the dicom dataset
dataset_train_gen = tf.data.Dataset.from_generator(dataset.create_train_dataset_generator,
                                              args=(imgBatchSize_train, ), output_types=(tf.float32,tf.float32),
                                              output_shapes=((None, roiSize, roiSize, 1), (None, roiSize, roiSize, 1))).prefetch(10)
 
dataset_val_gen = tf.data.Dataset.from_generator(dataset.create_val_dataset_generator,
                                              args=(imgBatchSize_val, ), output_types=(tf.float32,tf.float32),
                                              output_shapes=((None, roiSize, roiSize, 1), (None, roiSize, roiSize, 1))).prefetch(10)
 
dataset_test_gen = tf.data.Dataset.from_generator(dataset.create_test_dataset_generator,
                                              args=(imgBatchSize_test, ), output_types=(tf.float32,tf.float32),
                                              output_shapes=((None, roiSize, roiSize, 1), (None, roiSize, roiSize, 1))).prefetch(10)
 

In [ ]:
class Load_Image():
    
  def download(self):

    self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(array_metal, array_Or, test_size=0, random_state=42)


    self.train_dataset_shape = self.x_train.shape
    self.test_dataset_shape = self.x_test.shape


  def print_information(self):

    # Print informations about the DATA dataset.
    print("There is %d training samples, containing images with shape of: %dx%d and %d channel" 
          % (self.train_dataset_shape[0],self.train_dataset_shape[1],self.train_dataset_shape[2],self.train_dataset_shape[3]))
    print("Train variable shape:", end='')
    print(self.train_dataset_shape)

  def pre_process(self):

    # Reshape to have 1 channel (last dimension) and normalize
    self.x_train = np.expand_dims(self.x_train, axis=-1).astype(np.float32)
    self.x_test = np.expand_dims(self.x_test, axis=-1).astype(np.float32)
    self.y_train = np.expand_dims(self.y_train, axis=-1).astype(np.float32)
    self.y_test = np.expand_dims(self.y_test, axis=-1).astype(np.float32)

    # Normalize data
    ## https://stats.stackexchange.com/questions/185853/why-do-we-need-to-normalize-the-images-before-we-put-them-into-cnn
    self.x_train /= 255
    self.x_test /= 255
    self.y_train /= 255
    self.y_test /= 255


    self.train_dataset_shape = self.x_train.shape
    self.test_dataset = self.x_test.shape

  def train_train(self):

    self.x_train_noise = self.x_train 
    self.x_test_noise = self.x_test 
 
  
  def create_dataset_iterable(self):

    self.train_dataset = tf.data.Dataset.from_tensor_slices((self.x_train_noise, self.y_train))
    self.test_dataset = tf.data.Dataset.from_tensor_slices((self.x_test_noise, self.y_test))
      
  def shuffle_dataset(self, dataset_size):
    
    return self.train_dataset.shuffle(dataset_size)

In [ ]:
#Novo!!
class autoencoder_model(tf.keras.Model):

  def __init__(self):
      
      super(autoencoder_model, self).__init__()    

      self.n_filters = 32
      self.img_input_shape = (180, 180, 1) # nRows X nCols X nChannels
      
      self.encoder = self.build_encoder()
      self.decoder = self.build_decoder()
      self.autoencoder = self.build_autoencoder()


  def build_encoder(self):
      
      Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='valid', activation=None, strides=1, use_bias=True)      
      
      imgBatch = tf.keras.layers.Input(shape=self.img_input_shape)
      
      Conv2D_1 = Conv2D(filters=1*self.n_filters, kernel_size=5)(imgBatch)
      ReLU_1 = tf.keras.layers.ReLU()(Conv2D_1)
       
      Conv2D_2 = Conv2D(filters=1*self.n_filters, kernel_size=5)(ReLU_1)
      ReLU_2 = tf.keras.layers.ReLU()(Conv2D_2)
       
      Conv2D_3 = Conv2D(filters=1*self.n_filters, kernel_size=5)(ReLU_2)
      ReLU_3 = tf.keras.layers.ReLU()(Conv2D_3)
       
      Conv2D_4 = Conv2D(filters=1*self.n_filters, kernel_size=5)(ReLU_3)
      ReLU_4 = tf.keras.layers.ReLU()(Conv2D_4)
       
      Conv2D_5 = Conv2D(filters=1*self.n_filters, kernel_size=5)(ReLU_4)
      ReLU_5 = tf.keras.layers.ReLU()(Conv2D_5)
      
      model = tf.keras.Model(inputs=imgBatch, outputs=[imgBatch, ReLU_2, ReLU_4, ReLU_5], name="encoder")

      return model

  def build_decoder(self):
      '''
      https://medium.com/apache-mxnet/transposed-convolutions-explained-with-ms-excel-52d13030c7e8
      https://stackoverflow.com/questions/53654310/what-is-the-difference-between-upsampling2d-and-conv2dtranspose-functions-in-ker
      '''
      
      Conv2DTranspose = functools.partial(tf.keras.layers.Conv2DTranspose, padding='valid', activation=None, strides=1, use_bias=True)
      ResAdd = tf.keras.layers.Add()
      ResWeightedAdd = WeightedAdd(omega=0.5)
      
      Residual1 = tf.keras.layers.Input(shape=self.img_input_shape)
      Residual2 = tf.keras.layers.Input(shape=self.encoder.layers[4].output.shape[1::])   # ReLU_2
      Residual3 = tf.keras.layers.Input(shape=self.encoder.layers[8].output.shape[1::])   # ReLU_4

      LatentIn  = tf.keras.layers.Input(shape=self.encoder.layers[-1].output.shape[1::])
         
      # Upscaling convolutions (inverse of encoder)
      Conv2DT_1 = Conv2DTranspose(filters=1*self.n_filters, kernel_size=5) (LatentIn)
      Add_1 = tf.keras.layers.Add()([Conv2DT_1, Residual3])
      ReLU_1 =  tf.keras.layers.ReLU()(Add_1)
       
      Conv2DT_2 = Conv2DTranspose(filters=1*self.n_filters, kernel_size=5)(ReLU_1)
      ReLU_2 = tf.keras.layers.ReLU()(Conv2DT_2)
       
      Conv2DT_3 = Conv2DTranspose(filters=1*self.n_filters, kernel_size=5)(ReLU_2)
      Add_2 = ResAdd([Conv2DT_3, Residual2])
      ReLU_3 = tf.keras.layers.ReLU()(Add_2)
       
      Conv2DT_4 = Conv2DTranspose(filters=1*self.n_filters, kernel_size=5)(ReLU_3)
      ReLU_4 = tf.keras.layers.ReLU()(Conv2DT_4)
       
      Conv2DT_5 = Conv2DTranspose(filters=1, kernel_size=5)(ReLU_4)
      Add_3 = ResWeightedAdd([Conv2DT_5, Residual1])
      ReLU_5 = tf.keras.layers.ReLU()(Add_3)

      model = tf.keras.Model(inputs=[Residual1, Residual2, Residual3, LatentIn], outputs=ReLU_5, name="decoder")
       
      return model
  
  def build_autoencoder(self):
        
      imgBatch = tf.keras.layers.Input(shape=self.img_input_shape)
      
      Residual1, Residual2, Residual3, LatentIn = self.encoder(imgBatch)
      
      model = tf.keras.Model(imgBatch, self.decoder(inputs=[Residual1, Residual2, Residual3, LatentIn]), name="autoencoder")
      
      return model

  def call(self, x):
      
      latent_space = self.encoder(x)
      img_recon = self.autoencoder(x)
      
      return img_recon

In [ ]:
class WeightedAdd(tf.keras.layers.Layer):
    '''
    https://datascience.stackexchange.com/questions/56171/how-to-merge-two-cnn-deep-learning-model-using-weighted-sum-and-weighted-product
    ''' 
    
    def __init__(self, omega, **kwargs):
        
        super(WeightedAdd, self).__init__(**kwargs)
        
        self.omega = 0.5
        
    def call(self, model_outputs):
        
        return self.omega * model_outputs[1] + (1 - self.omega) * model_outputs[0]

In [ ]:
class perceptualModel(tf.keras.models.Model):
  def __init__(self, perceptual_layers):
    super(perceptualModel, self).__init__()
    self.perceptual_layers = perceptual_layers
    self.vgg = self.vgg_layers(perceptual_layers)
    self.vgg.trainable = False

  def vgg_layers(self, layer_names):
    """ Creates a vgg model that returns a list of intermediate output values."""
    
    # Load our model. Load pretrained VGG, trained on imagenet data
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    
    outputs = [vgg.get_layer(name).output for name in layer_names]

    model = tf.keras.Model([vgg.input], outputs)

    return model

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    perceptual_outputs = self.vgg(preprocessed_input)

    perceptual_dict = {perceptual_name:value
                  for perceptual_name, value
                  in zip(self.perceptual_layers, perceptual_outputs)}
    
    return perceptual_dict

In [ ]:
perceptual_layers = ['block3_conv1', 
                     'block3_conv2', 
                     'block3_conv3',
                     'block3_conv4']
                     
perceptualNet = perceptualModel(perceptual_layers)

In [ ]:
def loss_function(img_recon, img_without_noise):

    # MSE loss
    loss_MSE = tf.math.reduce_mean(tf.square(img_recon - img_without_noise),axis=(1,2))

    # Perceptual loss
    vgg_recon_out = perceptualNet(tf.tile(img_recon,(1,1,1,3)))
    vgg_without_noise_out = perceptualNet(tf.tile(img_without_noise,(1,1,1,3)))

    perceptual_loss = tf.add_n([tf.reduce_mean((vgg_recon_out[name]-vgg_without_noise_out[name])**2) for name in vgg_recon_out.keys()])
    perceptual_loss /= len(perceptual_layers)

    # Combination of both losses
    loss = loss_MSE + 1.17e-05 * perceptual_loss

    return loss

In [ ]:
def create_optimizer(learning_rate):

  return tf.keras.optimizers.Adam(learning_rate) 

In [ ]:
@tf.function
def train_step(autoencoder, img_with_noise , img_without_noise, optimizer):

  with tf.GradientTape() as tape:
        
    # Feed images into autoencoder
    img_recon = autoencoder(img_with_noise)
        
    # Calc the loss
    loss = loss_function(img_recon, img_without_noise)

    ### Backpropagation ###
    # Get the gradients
    grads = tape.gradient(loss, autoencoder.trainable_variables)

    # Update the weights
    optimizer.apply_gradients(zip(grads, autoencoder.trainable_variables))

  return loss

In [ ]:
import os
MODEL_SAVE_PATH = "."
EPS = 1e-4

def train_model(autoencoder, train_dataset, validation_dataset, num_epochs, batch_size, learning_rate, patience=25, tol=EPS, save_folder=MODEL_SAVE_PATH):

  optimizer = create_optimizer(learning_rate)

  train_loss_history = []
  validation_loss_history = []

  metrics_names = ['train_loss','val_loss'] 
  
  patience_counter = 0
  min_loss = None

  # Loop on each epoch
  for epoch in range(num_epochs):

    print("\nepoch {}/{} ".format(epoch+1,num_epochs), end="")

    epoch_train_loss = []
    for idX, (batch_x, batch_y) in enumerate(train_dataset):
      # Train the model
      train_loss = train_step(autoencoder, batch_x, batch_y, optimizer)

      values=[('train_loss',train_loss)]

      # progBar.update(idX*batch_size, values=values) 

      train_loss_history.append(tf.math.reduce_mean(train_loss))

      epoch_train_loss.append(tf.math.reduce_mean(train_loss).numpy())

    # Loop on each batch of test dataset for validation
    epoch_val_loss = []
    for batch_x, batch_y in validation_dataset:

      # Foward image through the network
      img_recon = autoencoder(batch_x)

      # Calc the loss
      val_loss = loss_function(img_recon, batch_y)

      validation_loss_history.append(tf.math.reduce_mean(val_loss))

      epoch_val_loss.append(tf.math.reduce_mean(val_loss).numpy())

    val_loss_reduced_mean = np.mean(epoch_val_loss)
    if min_loss is None or abs(val_loss_reduced_mean - min_loss) > tol:
      min_loss = val_loss_reduced_mean
      autoencoder.save_weights(os.path.join(save_folder, 'model_checkpoint'))
      patience_counter = 0
    else:
      patience_counter +=1 


    print(" Train loss:", epoch_train_loss[-1], "| Val. loss:", epoch_val_loss[-1])

    if patience_counter > patience:
      print(f"STOPING AT EPOCH {epoch}: PATIENCE EXCEEDED.")
      num_epochs = epoch + 1
      break
  
  train_loss = [train_loss_history[x].numpy() for x in range(0,len(train_loss_history),len(train_loss_history)//num_epochs)]
  val_loss = [validation_loss_history[x].numpy() for x in range(0,len(validation_loss_history),len(validation_loss_history)//num_epochs)]


  return train_loss, val_loss

In [ ]:
autoencoder = autoencoder_model()
autoencoder.encoder.summary()
autoencoder.decoder.summary()

In [ ]:
# Training hyperparameters
num_epochs = 500
batch_size = 16
learning_rate = 2e-4
patience = 20  
tol = 1e-3
#save_folder = "./checkpoint"
# Train the model 
train_loss, val_loss = train_model(
    autoencoder, 
    dataset_train_gen,
    dataset_val_gen,
    num_epochs,
    batch_size,
    learning_rate,
    patience=patience,
    tol=tol,
    save_folder=save_folder
)

In [ ]:
# Loading a saved model - Rodar quando tiver model_checkpoint salvo!
autoencoder.load_weights(os.path(save_folder, "model_checkpoint"))

In [ ]:
# Plot loss results

plt.figure()
print(train_loss)
print(val_loss)
plt.plot(train_loss, label="train_loss")
plt.plot(val_loss, label="val_loss")
plt.title("Results")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="uppdataset_test_gener right")

In [ ]:
import os
from os.path import join

N = 1

image_save_folder = "./image_save"

if not os.path.exists(image_save_folder):
  os.mkdir(image_save_folder)

for batch_x, batch_y in dataset_test_gen:

  for i, (img_metal, img_or) in enumerate(zip(batch_x, batch_y)):
    

    print(tf.reshape(img_metal, (1, *img_metal.shape)))
    img_recon = autoencoder(tf.reshape(img_metal, (1, *img_metal.shape)))

    print(img_recon.numpy().squeeze())
    plt.figure(figsize=(5, 5))
    ax_metal = plt.gca()
    ax_metal.imshow(img_metal.numpy().squeeze(),'gray')
    ax_metal.set_title("Metal", fontsize=15)
    
    plt.figure(figsize=(5, 5))
    ax_orig = plt.gca()
    ax_orig.imshow(img_or.numpy().squeeze(),'gray')
    ax_orig.set_title("Original", fontsize=15)
    
    plt.figure(figsize=(5, 5))
    ax_rec = plt.gca()
    ax_rec.imshow(img_recon.numpy().squeeze(),'gray')
    ax_rec.set_title("Reconstructed", fontsize=15)
  


    plt.imsave(join(image_save_folder, f"{i}_SinoCorrigido.png"), img_recon.numpy().squeeze(), cmap='gray')


    if i > N:
      break
    
    plt.savefig(join(image_save_folder, ""), cmap='gray')
  break
